In [ ]:
# リスト3.3.1 Elasticsearch サーバーへの接続
# ----------------------------------------------------------------------------------
# 目的:
#   ・Pythonクライアント（elasticsearch-py）から Elasticsearch クラスタへ接続し、
#     ノード/バージョン等のメタ情報を取得する最小例。
#
# 事前条件（理論・運用メモ）:
#   ・サーバ: Elasticsearch が起動済みであり、接続先（ホスト/ポート/認証方式）が把握できること。
#   ・クライアント: elasticsearch-py のメジャー版と ES 本体の版は整合させる（例: ES 8.x ↔ クライアント 8.x）。
#   ・セキュリティ: ES 8.x では TLS/認証が既定で有効な構成が多い。
#       - ローカル開発の平文HTTPなら  http://localhost:9200
#       - 基本認証なら              basic_auth=("elastic","<password>")
#       - APIキーなら                api_key=("<id>","<api_key>") あるいは "<id>:<api_key>"
#       - Elastic Cloud なら         cloud_id="<cloud id>"
#     ※ 本スニペットは“最小”を保つため引数を省略（= 既定の http://localhost:9200 を想定）している。
#   ・可観測性: 初回は es.ping() で疎通を確認 → 問題なければ es.info() で詳細を取得、の順が安全。
#
# es.info(pretty=True) の意味:
#   ・クラスタ名/UUID/バージョン/ビルド情報/ノード情報などを返す（dict）。
#   ・pretty=True はサーバへのクエリ文字列 ?pretty=true を付ける指定（整形JSON）。Pythonクライアント側では
#     返り値は dict のため、表示整形は print(json.dumps(..., indent=2)) などで行うのが一般的。
# ----------------------------------------------------------------------------------

from elasticsearch import Elasticsearch

# クライアント生成（最小例）
# - 引数なし: 既定で "http://localhost:9200" に接続を試みる。
# - 実運用では接続先/認証/TLSを明示すること（以下に例をコメントで併記）。
es = Elasticsearch()

# 例1) 明示ホスト（平文HTTP・ローカル）
# es = Elasticsearch("http://localhost:9200")

# 例2) 基本認証 + TLS（自己署名証明書を無視するなら verify_certs=False を明示）
# es = Elasticsearch(
#     "https://localhost:9200",
#     basic_auth=("elastic", "your-password"),
#     verify_certs=False
# )

# 例3) Elastic Cloud（Cloud ID + API Key）
# es = Elasticsearch(
#     cloud_id="your-cloud-id",
#     api_key=("api_key_id", "api_key_secret")
# )

# 例4) タイムアウト/リトライ/コネクションプールの制御（大規模環境向け）
# es = Elasticsearch(
#     ["http://es1:9200", "http://es2:9200"],
#     request_timeout=10, retries=2, sniff_on_start=False
# )

# サーバ情報の取得
# - 疎通や認証に失敗すると例外が送出される（ConnectionError/AuthenticationExceptionなど）。
# - pretty=True はサーバ側整形。Python側で整形表示したい場合は json.dumps を用いる。
info = es.info(pretty=True)
print(info)

# 補足: 疎通確認の最小チェック（必要なら有効化）
# from elasticsearch import ConnectionError
# try:
#     if es.ping():
#         print("Ping OK")
#     else:
#         print("Ping NG（クラスタ不応答の可能性）")
# except ConnectionError as e:
#     print("接続エラー:", e)

# 補足: 明示的に最新インデックス状態を反映させたい（検証手順でPUT直後に検索する等）
# es.indices.refresh(index="_all")